In [ ]:
import torch
import random
import torchvision
import numpy as np
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import transforms

from avalanche.training.supervised import LwF
from avalanche.logging import InteractiveLogger, WandBLogger
from avalanche.benchmarks.datasets import CIFAR100
from avalanche.benchmarks.generators import nc_benchmark
from avalanche.benchmarks.utils import AvalancheDataset
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import ExperienceAccuracy, ExperienceLoss, ExperienceForgetting, ExperienceTime, EpochAccuracy


In [ ]:
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
cudnn.enabled = False
cudnn.deterministic = True

In [ ]:
train_set = CIFAR100('../Dataset/CIFAR100', train=True, download=True)
test_set = CIFAR100('../Dataset/CIFAR100', train=False, download=True)

transforms_group = dict(
       train=(
       transforms.Compose(
              [
              transforms.ToTensor(),
              ]
       ),
       None,
       ),
       eval=(
       transforms.Compose(
              [
              transforms.ToTensor(),
              ]
       ),
       None,
       )
)

train_set = AvalancheDataset(train_set, transform_groups=transforms_group, initial_transform_group="train")
test_set = AvalancheDataset(test_set, transform_groups=transforms_group, initial_transform_group="eval")

In [ ]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

num_class = 100
incremental = 10

scenario = nc_benchmark(train_dataset=train_set,
                        test_dataset=test_set,
                        n_experiences=incremental,
                        task_labels=False,
                        seed=seed,
                        shuffle=False
                        )

model = torchvision.models.resnet18(pretrained=False, num_classes=num_class)
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=1e-3)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
interactive_logger = InteractiveLogger()
wandb_logger = WandBLogger(project_name="CL-Test", run_name="iCaRL-CIFAR10")
eval_plugin = EvaluationPlugin(
    EpochAccuracy(),
    ExperienceAccuracy(),
    ExperienceLoss(),
    ExperienceForgetting(),
    ExperienceTime(),
    loggers=[interactive_logger, wandb_logger])

## LwF

In [ ]:
train_batch = 128
eval_batch = 128
epoch = 10
lwf_alpha = [0, 0.5, 1.333, 2.25, 3.2]

strategies = LwF(model, optimizer, criterion, alpha=lwf_alpha, temperature=1, train_epochs=epoch, device=device, train_mb_size=train_batch, eval_mb_size=eval_batch, evaluator=eval_plugin)

In [ ]:
print("Starting experiment...")
results = []

for experience in scenario.train_stream:
    print("Start training on experience ", experience.current_experience)
    strategies.train(experience)
    print("End training on experience ", experience.current_experience)
    print("Computing accuracy on the test set")
    results.append(strategies.eval(scenario.test_stream[:]))